<a href="https://colab.research.google.com/github/habataku/Animex/blob/master/Studyplus%E3%81%84%E3%81%84%E3%81%AD%E7%89%B9%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最初に実行

In [13]:
#@title クラス定義 最初に実行 {display-mode:"form"}
import requests
import json
import re
import datetime
import pprint
from IPython.display import display,Markdown

class IStudyPlusUser:
  def __init__(self,json):
    self.data=json

  def followee(self):
    json=requests.get("https://api.studyplus.jp/2/users?followee="+self.data["username"]+"&page=1&per_page=100&include_recent_record_seconds=t").json()
    followees=json["users"]
    if json["page"]<json["total_page"]:
      for i in range(2,json["total_page"]+1):
        followees += requests.get("https://api.studyplus.jp/2/users?followee="+self.data["username"]+"&page="+str(i)+"&per_page=100&include_recent_record_seconds=t").json()["users"]
    return followees
  
  def follower(self):
    json=requests.get("https://api.studyplus.jp/2/users?follower="+self.data["username"]+"&page=1&per_page=100&include_recent_record_seconds=t").json()
    followers=json["users"]
    if json["page"]<json["total_page"]:
      for i in range(2,json["total_page"]+1):
        followers += requests.get("https://api.studyplus.jp/2/users?follower="+self.data["username"]+"&page="+str(i)+"&per_page=100&include_recent_record_seconds=t").json()["users"]
    return followers

class IStudyPlus:
  def __init__(self):
    self.api_root="https://api.studyplus.jp/2"
    result=requests.get("https://www.studyplus.jp/")
    self.web_session=result.cookies.get("_studyplus-web_session")
    self.cookies={"_studyplus-web_session":self.web_session,"AMP_TOKEN":"%24NOT_FOUND"}
    self.csrf_token=re.search(r'<meta name="csrf-token" content="(.{88})"',result.text).group(1)
    self.headers = {
            'authority': 'www.studyplus.jp',
            'accept': 'application/json, text/html, */*',
            'accept-language': 'ja,en-US;q=0.9,en;q=0.8',
            'cache': 'no-cache',
            'content-type': 'application/json',
            'origin': 'https://www.studyplus.jp',
            'referer': 'https://www.studyplus.jp',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1 Edg/112.0.0.0',
            'x-csrf-token': self.csrf_token
    }

  def login(self,username,password,remember_me=False):
    data={"username":username,"password":password,"remenber_me":remember_me}
    result=requests.post("https://www.studyplus.jp/api/auth",data=json.dumps(data),cookies=self.cookies,headers=self.headers)
    self.web_session=result.cookies.get("_studyplus-web_session")
    self.auth=result.cookies.get('auth')
    self.cookies={"_studyplus-web_session":self.web_session,"auth":self.auth,"AMP_TOKEN":"%24NOT_FOUND"}
    self.username=re.search('username%22%3A%22(.*)%22%2C%22',self.auth).group(1)
    self.access_token=re.search('access_token%22%3A%22(.*)%22%7D',self.auth).group(1)
    self.headers["Authorization"]="OAuth "+self.access_token
    return self

  def __del__(self):
    if self.username:
      requests.post('https://www.studyplus.jp/auth',data="_method=delete&authenticity_token="+self.csrf_token,headers={
              'authority': 'www.studyplus.jp',
              "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
              "accept-language": "ja,en-US;q=0.9,en;q=0.8",
              "cache-control": "max-age=0",
              "content-type": "application/x-www-form-urlencoded",
              "sec-ch-ua": "\"Google Chrome\";v=\"113\", \"Chromium\";v=\"113\", \"Not-A.Brand\";v=\"24\"",
              "sec-ch-ua-mobile": "?0",
              "sec-ch-ua-platform": "\"Windows\"",
              "sec-fetch-dest": "document",
              "sec-fetch-mode": "navigate",
              "sec-fetch-site": "same-origin",
              "sec-fetch-user": "?1",
              "upgrade-insecure-requests": "1",
      })

  def me(self):
    if not self.username:
      return
    result=requests.get(self.api_root+"/users/"+self.username,cookies=self.cookies,headers=self.headers)
    self.web_session=result.cookies.get("_studyplus-web_session")
    self.cookies={"_studyplus-web_session":self.web_session,"auth":self.auth,"AMP_TOKEN":"%24NOT_FOUND"}
    return IStudyPlusUser(result.json())

In [14]:
#@title ログイン {display-mode:"form"}
メールアドレス="habatakutori.fbi@gmail.com"#@param {type:"string"}
パスワード="ctuisipnNian0972"#@param {type:"string"}
sp=IStudyPlus().login(メールアドレス,パスワード)
me=sp.me()

# **自動いいね**

In [ ]:
# @title 自分のタイムラインに指定した個数いいね {display-mode:"form"}
種類="\u5B9A\u671F\u30C6\u30B9\u30C8 \u6210\u7E3E\u30A2\u30C3\u30D7"#@param ["フォロー中","定期テスト 成績アップ"]
if 種類=="フォロー中":
  timeline_url="followee"
elif 種類=="定期テスト 成績アップ":
  timeline_url="study_goal/test"
json=requests.get("https://api.studyplus.jp/2/timeline_feeds/"+timeline_url,cookies=sp.cookies,headers=sp.headers).json()
個数_30個単位=300#@param {type:"number"}

for i in range(0,個数_30個単位//30):
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(list(record.values())[0]["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/"+timeline_url+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

In [ ]:
#@title 達成目標idから指定した個数いいね {display-mode:"form"}
達成目標id="college-578"# @param {type:"string"}
json=requests.get("https://api.studyplus.jp/2/timeline_feeds/study_goal/"+達成目標id,cookies=sp.cookies,headers=sp.headers).json()
個数_30個単位=300#@param {type:"number"}

for i in range(0,個数_30個単位//30):
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(list(record.values())[0]["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/study_goal/"+達成目標id+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

In [ ]:
#@title 自分の達成目標id一覧取得 {display-mode:"form"}
md="|達成目標|id|\n|--|--|\n"
for val in me.data['study_goals']:
  md+=f"|{val['label']}|{val['key']}|\n"
display(Markdown(md))

|達成目標|id|
|--|--|
|英検 準1級|english-eiken-jun1|
|広島大学|college-653|
|大阪大学|college-524|
|京都大学|college-491|
|神戸大学|college-578|
|千葉大学|college-147|
|名古屋大学|college-428|
|北海道大学|college-5|


In [ ]:
# @title 人を検索して全いいね
検索元="フォロワー"#@param ["フォロワー","フォロー中","id指定"]
名前またはid="1234567890"#@param {type:"string"}

if 検索元=="フォロワー":
  user=next(filter(lambda x:x["nickname"]==名前またはid or x["username"]==名前またはid,me.follower()),None)
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+user["username"],cookies=sp.cookies,headers=sp.headers).json()
elif 検索元=="フォロー中":
  user=next(filter(lambda x:x["nickname"]==名前またはid or x["username"]==名前またはid,me.followee()),None)
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+user["username"],cookies=sp.cookies,headers=sp.headers).json()
else:
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+名前またはid,cookies=sp.cookies,headers=sp.headers).json()

for i in range(0,2000):
  count=0
  for record in json["feeds"]:
    try:
      record.pop("feed_type")
      body=list(record.values())[0]
      if body["if_you_like"]:
        count+=1
        continue
      requests.post("https://api.studyplus.jp/2/timeline_events/"+str(body["event_id"])+"/likes/like",cookies=sp.cookies,headers=sp.headers).json()
    except:
      continue
  if count==len(json["feeds"]) and i>100:
    break
  json=requests.get("https://api.studyplus.jp/2/timeline_feeds/user/"+user["username"]+"?until="+json["next"],cookies=sp.cookies,headers=sp.headers).json()

# **ユーザー情報**

In [ ]:
#@title 自分のフォロワー/フォロー中一覧 {display-mode:"form"}
モード="\u30D5\u30A9\u30ED\u30FC\u4E2D"#@param ["フォロワー","フォロー中"]
if モード=="フォロワー":
  userlist=me.follower()
elif モード=="フォロー中":
  userlist=me.followee()

md="|名前|id|\n|--|--|\n"
for val in userlist:
  md+=f"|{val['nickname']}|{val['username']}|\n"
display(Markdown(md))

In [ ]:
#@title 知り合いの知り合い一覧 {display-mode:"form"}
取得する人="\u30D5\u30A9\u30ED\u30EF\u30FC"#@param ["フォロワー","フォロー中"]
ユーザー名="shu"#@param {type:"string"}
モード="\u30D5\u30A9\u30ED\u30EF\u30FC"#@param ["フォロワー","フォロー中"]

if 取得する人=="フォロワー":
  user=next(filter(lambda x:x["nickname"]==ユーザー名,me.follower()),None)
elif 取得する人=="フォロー中":
  user=next(filter(lambda x:x["nickname"]==ユーザー名,me.followee()),None)
print(f'{取得する人}の{ユーザー名} さんの{モード}一覧:')
if モード=="フォロワー":
  userlist=IStudyPlusUser(user).follower()
elif モード=="フォロー中":
  userlist=IStudyPlusUser(user).followee()

md="|名前|id|\n|--|--|\n"
for val in userlist:
  md+=f"|{val['nickname']}|{val['username']}|\n"
display(Markdown(md))

In [ ]:
#@title ユーザーidから情報を取得 {display-mode:"form"}
username="97a7afe10247462a99d6e858ede46674"#@param {type:"string"}
user=IStudyPlusUser(requests.get("https://api.studyplus.jp/2/users/"+username).json())
print(user.data["nickname"]+"さんの情報:")
pprint.pprint(user.data)

In [ ]:
# @title 没
# import requests
# from dataclasses import dataclass,fields
# from typing import Optional

# @dataclass
# class IStudyPlusUserProfile:
#   user_id: int
#   username: str
#   user_image_url: str
#   nickname: str
#   badge_type: str
#   user_organizations: list
#   disable_reply: bool
#   allow_send_image_message: bool
#   disable_profile_view: bool
#   goal: str
#   user_relationship_id: int
#   user_relationship_status: str
#   target_user_relationship_id: int
#   target_user_relationship_status: str
#   location_code: int
#   location: str
#   job_code: int
#   job: str
#   job_grade: int
#   follower_policy: str
#   feed_visibility: str
#   desired_departments: list
#   mute: bool
#   fav: bool
#   feed_is_visible: bool
#   study_goals: list

#   def followee(self):
#     json=requests.get("https://api.studyplus.jp/2/users?followee="+self.username+"&page=1&per_page=100&include_recent_record_seconds=t").json()
#     followees=json["users"]
#     if json["page"]<json["total_page"]:
#       for i in range(2,json["total_page"]+1):
#         followees |= requests.get(self.api_root+"/users?followee="+self.username+"&page="+i+"&per_page=100&include_recent_record_seconds=t").json()
#     followeeprofiles=[]
#     print(followees)
#     for follower in followees:
#       for field in fields(IStudyPlusRelateUser):
#         if field.name not in follower:
#           follower[field.name]=""
#       followeeprofiles.append(IStudyPlusRelateUser(**follower))
#     return followeeprofiles

# @dataclass
# class IStudyPlusUser(IStudyPlusUserProfile):
#   follow_count: int
#   follower_count: int
#   follow_requesting_count: int
#   biography: str
#   tags: list
#   educational_background: dict
#   material_count: int
#   premium: bool
#   book_user: bool
#   use_drill: bool
#   studyplus_confirmed: bool
#   studylog_confirmed: bool
#   record_count: int
#   record_hours_this_month: int
#   record_hours_last_month: int
#   total_record_hours: int
#   has_study_challenge: bool
#   sex: str
#   birthyear: int
#   birthmonth: int
#   birthdate: int
#   occupation: str
#   study_achievements: list
#   examination_count: int
#   email_is_confirmed: bool
#   learning_material_review_count: int
#   learning_material_review_like_count: int
#   desired_department_visibility: str
#   sex_visibility: str
#   birthyear_visibility: str
#   birthdate_visibility: str
#   location_visibility: str
#   job_visibility: str
#   occupation_visibility: str
#   study_goals_visibility: str
#   limited_message: bool

# @dataclass
# class IStudyPlusRelateUser(IStudyPlusUserProfile):
#   recent_record_seconds:int
